# Fluid modeling

PorePy supports the introduction of multiphase, multicomponent fluids into its models.

This tutorial focuses on relevant mixin-methods and two approaches to modeling of fluid properties:

- heuristic approach using functions in the form of constitutive laws
- Equation of State (EoS) -based approach using the respective object for computations

### Table of contents

1. [Basics and Default Behavior in PorePy](#basics-and-default-behavior-in-porepy)
2. Modeling component- and phase-context
3. Modeling of fluid properties
    - Heuristic approach (default)
    - Equation-of-state-approach
4. Overriding individual properties
5. EoS approach
    - The EquationOfState class
    - Defining dependencies of phase properties
6. Creating individual component-contexts for phases
7. (?) Related degrees of freedom

## Basics and Default Behavior in PorePy

Fluids can be defined in terms of two contexts: components and phases.

While the number of components is something the modeler choses naturally, the number (and more importantly type) of phases is
an unknown number in the general thermodynamic sense.
In PorePy, the modeler must define the (maximal) number and types of phases expected in the course of simulations.

The modeling can then be roughly performed in 3 steps:

1. Defining the components in the fluid (water, CO2, ...)
2. Defining the expected phases (1 gas phase, 2 liquid phases for example)
3. Introducing a thermodynamic model for fluid properties (densities, enthalpies, ...)

> **Note on distinction between fluid, phase and component properties:**
> 
> It makes no sense to talk about the density of a component, say water.
> Liquid and gaseous water have greatly different densities.
> I.e., extensive properties like density and enthalpy are always related to the phase.
> Multiple components can be present in a phase and influence the properties.
> Extensive properties of fluid (mixtures) (consisting of multiple components in multiple phases), depend in a clearly defined manner on the respective properties of present phases (usually a weighted sum of phase properties).

The default fluid in PorePy models consists of 1 component, and 1 phase containing that component.
In this simple case, the fluid properties are equal to the (single) phase properties.

The class [`FluidMixin`](../src/porepy/compositional/compositional_mixins.py#L1042) provides respective functionalities for creating a fluid, and is part of all models provided in PorePy.

Legacy implementations provides a simple customization of the single-component without major effort:


In [ ]:
import porepy as pp
from porepy.applications.material_values.fluid_values import water

model_params = {
    'material_constants': {
        'fluid': pp.FluidComponent(**water)
    }
}

model = pp.SinglePhaseFlow(model_params)

Above code creates the default single-phase, single-component fluid and is ready for simulations right away.
It is compatible with all constitutive laws found in the [fluid property library](../src/porepy/models/fluid_property_library.py).

Note however, that the `FluidMixin` is part of all models found in the `models` subpackage.
It comes with some general implementations of fluid properties, which are covered in a later section.
To override those general implementations, constitutive laws must be treated as pure mixins (as intended).

For customization, they must be on top in the MRO:

In [ ]:
class CustomSinglePhaseFlow(
    pp.constitutive_laws.FluidDensityFromPressureAndTemperature,
    pp.SinglePhaseFlow,
):
    """Just for demonstration purpose. Note that the base model has no notion of temperature."""
    